In [14]:
%pip install cassiopeia
%pip install mysql.connector

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
  Using cached PyMySQL-1.1.0-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.0-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cassiopeia as cass
from cassiopeia.data import Queue, Position, GameMode, Side, Lane, Tier
from cassiopeia.core import Summoner, Match
import pandas as pd
from cassiopeia import Champions, Champion
import connect_tools as ct
import time
import urllib.error as urlerror

In [8]:
cass.set_riot_api_key(key="RGAPI-9907fab6-fb67-4811-991a-a81783d6979e")

In [9]:
def df_summoners(entry, region, league):

    parameters = {"name_summoner" : entry.summoner.name, "riot_id_summoner" : entry.summoner.id, "region_summoner" : region, "usual_lane" : None, "league_summoner" : league}

    db_summoners = pd.DataFrame(parameters, index = [0])
    return db_summoners

def Found_summoners(region : str):
    """Create a dataframe of Challengers, Grand Masters and Master"""

    def Found_challengers(region):
        league = "Challenger"
        challenger = cass.get_challenger_league(queue=Queue.ranked_solo_fives, region=region)
        challengers_list = []
        for position, entry in enumerate(challenger):
            df = df_summoners(entry, region, league)
            challengers_list.append(df)
        return challengers_list
    
    def Found_grandmaster(region):
        league = "GrandMaster"
        grandmaster = cass.get_grandmaster_league(queue=Queue.ranked_solo_fives, region=region)
        grandmaster_list = []
        for position, entry in enumerate(grandmaster):
            df = df_summoners(entry, region, league)
            grandmaster_list.append(df)
        return grandmaster_list
    
    def Found_masters(region):
        league = "Master"
        master = cass.get_master_league(queue=Queue.ranked_solo_fives, region=region)
        masters_list = []
        for position, entry in enumerate(master):
            df = df_summoners(entry, region, league)
            masters_list.append(df)
        return masters_list

    
    challengers_list = Found_challengers(region)
    grandmasters_list = Found_grandmaster(region)
    masters_list = Found_masters(region)
    
    temp_chall = pd.concat(challengers_list, ignore_index= True)
    temp_grandmaster = pd.concat(grandmasters_list, ignore_index= True)
    temp_master = pd.concat(masters_list, ignore_index=True)

    summoners_df = pd.concat([temp_chall, temp_grandmaster, temp_master], ignore_index= True)
    summoners_df = summoners_df.drop_duplicates(subset="riot_id_summoner")
    summoners_df = summoners_df.drop_duplicates(subset="name_summoner")

    password = ""
    connexion = ct.Connect_Tools(password=password)
    connexion.insert_db(summoners_df, table_name="summoners")
    connexion.close_conn()

In [4]:
Found_summoners(region="EUW")

Making call: https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5
Making call: https://euw1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5
Making call: https://euw1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5


In [2]:
def s_c(p, columns):
    for i in p:
        if i.side == Side.red and i.lane == Lane.top_lane and columns == "red_top":
            return i.champion.name
        elif i.side == Side.red and i.lane == Lane.jungle and columns == "red_jun":
            return i.champion.name
        elif i.side == Side.red and i.lane == Lane.mid_lane and columns == "red_mid":
            return i.champion.name
        elif i.side == Side.red and i.lane == Lane.bot_lane and columns == "red_bot":
            return i.champion.name
        elif i.side == Side.red and i.lane == Lane.utility and columns == "red_sup":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.top_lane and columns == "blu_top":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.jungle and columns == "blu_jun":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.mid_lane and columns == "blu_mid":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.bot_lane and columns == "blu_bot":
            return i.champion.name
        elif i.side == Side.blue and i.lane == Lane.utility and columns == "blu_sup":
            return i.champion.name

def df_matchs(match, p, ref_id_summoner):
    parametres = {
        "red_top": s_c(p, "red_top"),
        "red_jun": s_c(p, "red_jun"),
        "red_mid": s_c(p, "red_mid"),
        "red_bot": s_c(p, "red_bot"),
        "red_sup": s_c(p, "red_sup"),
        "blu_top": s_c(p, "blu_top"),
        "blu_jun": s_c(p, "blu_jun"),
        "blu_mid": s_c(p, "blu_mid"),
        "blu_bot": s_c(p, "blu_bot"),
        "blu_sup": s_c(p, "blu_sup"),
        "win_side": match.blue_team.win,
        "ref_id_summoners" : ref_id_summoner,
        "riot_id_match": match.id
    }

    df = pd.DataFrame(parametres, index=[0])
    return df

def check_participants(p):
    try:
        for i in p:
            if i.lane == None:
                return False
            else:
                pass
    except KeyError:
        return False

password = ""
connexion = ct.Connect_Tools(password=password)

def get_matches(summoner, num_matches):
    match_dataframes = []
    count = 0
    for i , match in enumerate(summoner.match_history[1:num_matches]):

        vr = connexion.req_fetchone('SELECT `riot_id_match` FROM `matches` WHERE `riot_id_match` = %s', match.id)

        p = match.participants

        cp = check_participants(p)

        if vr != None:
            print("Match already in database...")
            continue
        if count >= 10:
            break
        if cp == False:
            print("Not all participants in match...")
            continue
        if match.start == False:
            print("Match not begin...")
            continue
        if match.mode != GameMode.classic:
            print("Not Classic Mode...")
            continue
        if match.queue != Queue.ranked_solo_fives:
            print("Not Ranked five...")
            continue
        if match.is_remake == True:
            print("Remake match...")
            continue

        ref_id_summoner = connexion.req_fetchone('SELECT `id_summoner` FROM `summoners` WHERE `riot_id_summoner`= %s',summoner.id)

        df_match = df_matchs(match, p, ref_id_summoner)
        match_dataframes.append(df_match)
        count += 1

    return match_dataframes
    
def main(id : str , region : str):
    try:
        summoner = Summoner(id = id , region=region)
        match_dataframes = get_matches(summoner, 50)

        df_main = pd.concat(match_dataframes, ignore_index= True)

        connexion.insert_db(dataframe = df_main, table_name= 'matches')
            
    except KeyError:
        print("Not enough game...")
    except ValueError:
        print("Empty Data...")

def all_main(region : str):
    all_sum = connexion.req_fetchall('SELECT `riot_id_summoner` FROM `summoners`')
    for i in all_sum:
        ref_is = connexion.req_fetchone('SELECT `id_summoner` FROM `summoners` WHERE `riot_id_summoner` = %s', i[0])
        count_m = connexion.req_fetchone('SELECT COUNT(*) FROM `matches` WHERE `ref_id_summoners` = %s', ref_is[0])
        if count_m[0] >= 10:
            print("Already 10 matches in database...")
            continue
        else:
            main(i[0], region)
    connexion.close_conn()

def exe_getmatches(region : str):
    while True:
        try:
            all_main(region)
            break
        except urlerror.HTTPError as httperror:
            print(httperror)
            print("You are disconnect from Riot server after an error. Trying to reconnect in 60 seconds...")
            time.sleep(60)

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée)

In [11]:
exe_getmatches("EUW")

Already 10 matches in database...
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/__PyBdAwGcL6D9ld-K8WkCA5JTnZKcZO3uCy2Bz0Kc6b8zU
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/ZQcAs_l9bMO-o8z6ghfepFZ-RiAUH9OAcZYTq55aU8_FMy94ZBp7J0uWADkk5Aabh26oe_ooSfUoUA/ids?start=0&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_6860495385
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_6859657329
Not all participants in match...
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_6859623956
Not Ranked five...
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_6858700001
Not all participants in match...
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_6858670039
Not all participants in mat

APIError: The Riot API experienced an internal error on the request. You may want to retry the request after a short wait or continue without the result. The received error was 500: "Internal server error"

In [4]:
def df_champions(champion):
    parametres = {
        "id_champion" : champion.id,
        "name_champ" : champion.name
    }
    
    df = pd.DataFrame(parametres, index=[0])
    return df
    
def champion_stats(region : str):
    cham_dataframe = []
    champions = Champions(region=region)
    for champion in champions:
        df_ch = df_champions(champion)
        cham_dataframe.append(df_ch)
    final_df = pd.concat(cham_dataframe, ignore_index=True)
    password = ""
    connexion = ct.Connect_Tools(password=password)
    connexion.insert_db(dataframe= final_df, table_name = 'champions_list')
    connexion.close_conn()

In [5]:
champion_stats("EUW")

Making call: https://ddragon.leagueoflegends.com/realms/euw.json
Making call: https://ddragon.leagueoflegends.com/cdn/14.5.1/data/en_GB/championFull.json
